<span style="background-color: green;">因为HDataFrame和VDataFrame使用均与DataFrame类似，所以这里展示VDataFrame的数据情况。</span>


# 1 读取预处理数据

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob'], address='local')
alice = sf.PYU('alice')
bob = sf.PYU('bob')

The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-20 14:22:15,029	INFO worker.py:1724 -- Started a local Ray instance.


准备需要预处理的数据并保存

In [2]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
data = pd.concat([iris.data, iris.target], axis=1)

# In order to facilitate the subsequent display,
# here we first set some data to None.
data.iloc[1, 1] = None
data.iloc[100, 1] = None

# Restore target to its original name.
data['target'] = data['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,NaN,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
import tempfile
from secretflow.data.vertical import read_csv as v_read_csv

# Vertical partitioning.
v_alice, v_bob = data.iloc[:, :2], data.iloc[:, 2:]

# Save to temprary files.
_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
v_alice.to_csv(alice_path, index=False)
v_bob.to_csv(bob_path, index=False)

读取数据

In [4]:
df = v_read_csv({alice: alice_path, bob: bob_path})

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


# 2 VDataFrame数据属性和方法及数据预处理

**Attributes:**

| [`partitions`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#id0) |                                                              |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| [`aligned`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#id1) |                                                              |
| [`columns`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.columns) | The column labels of the DataFrame.                          |
| [`dtypes`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.dtypes) | Return the dtypes in the DataFrame.                          |
| [`shape`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.shape) | Return a tuple representing the dimensionality of the DataFrame. |
| [`values`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.values) | Return a federated Numpy representation of the DataFrame.    |
| [`partition_columns`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.partition_columns) | Returns columns of each partition.                           |

**Methods:**

| [`count`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.count)([numeric_only]) | Count non-NA cells for each column.                          |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| [`index`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.index)() | Returns the index (row labels) of the DataFrame.             |
| [`sum`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.sum)([numeric_only]) | Return the sum of the values over the axis 0.                |
| [`min`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.min)([numeric_only]) | Return the min of the values over the axis 0.                |
| [`max`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.max)([numeric_only]) | Return the max of the values over the axis 0.                |
| [`mean`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.mean)([numeric_only]) | Return the mean of the values over the axis 0.               |
| [`var`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.var)([numeric_only]) | Return the var of the values over the axis 0.                |
| [`std`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.std)([numeric_only]) | Return the std of the values over the axis 0.                |
| [`sem`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.sem)([numeric_only]) | Return the standard error of the mean over the axis 0.       |
| [`skew`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.skew)([numeric_only]) | Return the skewness over the axis 0.                         |
| [`kurtosis`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.kurtosis)([numeric_only]) | Return the kurtosis over the requested axis.                 |
| [`quantile`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.quantile)([q, numeric_only]) | Returns values at the given quantile over axis 0.            |
| [`mode`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.mode)([numeric_only, dropna]) | Return the mode of the values over the axis 0.               |
| [`value_counts`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.value_counts)(*args, **kwargs) | Return a Series containing counts of unique values.          |
| [`isna`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.isna)() | "Detects missing values for an array-like object. Same as pandas.DataFrame.isna Returns DataFrame: Mask of bool values for each element in DataFrame that indicates whether an element is an NA value. |
| [`replace`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.replace)(*args, **kwargs) | Replace values given in to_replace with value.               |
| [`astype`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.astype)(dtype[, copy, errors]) | Cast object to a specified dtype `dtype`.                    |
| [`copy`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.copy)() | Shallow copy of this dataframe.                              |
| [`drop`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.drop)([labels, axis, index, columns, level, ...]) | Drop specified labels from rows or columns.                  |
| [`fillna`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.fillna)([value, method, axis, inplace, ...]) | Fill NA/NaN values using the specified method.               |
| [`to_csv`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.to_csv)(fileuris, **kwargs) | Write object to a comma-separated values (csv) file.         |
| [`iloc`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.iloc)(index) | Integer-location based indexing for selection by position.   |
| [`rename`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.rename)([mapper, index, columns, axis, copy, ...]) | See [`pandas.DataFrame.rename()`](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename) |
| [`pow`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.pow)(*args, **kwargs) | Gets Exponential power of dataframe and other, element-wise (binary operator pow). |
| [`round`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.round)(decimals) | Round the DataFrame to a variable number of decimal places.  |
| [`select_dtypes`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.select_dtypes)(*args, **kwargs) | Returns a subset of the DataFrame's columns based on the column dtypes. |
| [`subtract`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.subtract)(*args, **kwargs) | Gets Subtraction of dataframe and other, element-wise (binary operator sub). |
| [`apply_func`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.apply_func)(func, *[, nums_return]) | Apply any function inside the dataframe actor.               |
| [`partition_shape`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.partition_shape)() | Return shapes of each core.                                  |
| [`to_pandas`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.to_pandas)() | Convert myself to pandas type.                               |
| [`groupby`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.groupby)(compute_device, by) | Group the VDataFrame by the specified columns.               |
| [`get_numeric_cols_from_df`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.get_numeric_cols_from_df)(cols) | Get columns from VDataFrame.                                 |
| [`get_flat_column`](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/source/secretflow.data.data.vertical#secretflow.data.vertical.VDataFrame.get_flat_column)(col_name) | From the VDataFrame, extract a single column in flattened numpy array |